# Substitution Ciphers

A substitution cipher is a method of encrypting in which every plaintext character (or group of characters) is replaced with a different ciphertext symbol. The receiver deciphers the text by performing the inverse substitution.

<img src="images/substitution-cipher.png" width=500px>
<!-- ![substitution](images/substitution-cipher.png) -->

- Substitution can consider single characters (simple substitution cipher) but also group of characters (e.g., pair, triplets, and so on).
- Alphabet simple substitution Ciphers admits $26!\sim 10^{26}\sim 2^{88}$ possible encoding rules (not easy to try them all). Assuming 1ns for each try, it would take $>10^9$years.
- However, substitution does not alter the statistics so plaintext can be deduced by analyzing the frequency distribution of the ciphertext.

In [1]:
%matplotlib widget
import matplotlib.pyplot as plt

import numpy as np
from scipy import io

## Caesar Cipher

The method is named after Julius Caesar, who used it in his private correspondence. Each letter in the plaintext is replaced by a letter some fixed number of positions down the alphabet.
- Same characters for plaintext and ciphertext.
- Very simple encoding rule. Only 26 possibilities!

<img src="images/caesar-cipher.png" width=500px>

Two easy way to break the cipher:
- **Brute force**: Since alphabet is 26 letters long only 26 shifts are possible you can try all possibilities and check them all.
- **Frequency analysis**: Knowing what is the frequency of letters (e.g., in English, letters «e», «t», «a», «i» are more common than others), it is possible to infer what shift was used by oserving the frequency of the characters in the ciphertext.

### Decoder

As first thing, we need to define the decoding procedure.

Decoder needs two pieces of information:
- the **alphabet**: it may seem trivial but substitution cipher assumes the decoder knows both plain and cipher alphabet. In case of Caesar cipher they are the same so we just need to define one.
- the **shift** (the key): given the alphabet, the Caesar cipher is completely defined by the shift that must be applied to the plain alphabet to obtain the cipher alphabet.

We choose to implement the decoder as a function that takes the `ciphertext`a and the `shift` as input and returns the `plaintext`. The alphabet is hard-coded in the function.

In [22]:
def caesar_decoding(ciphertext, shift=0):
    '''
    Decode a ciphertext encrypted with a Caesar Cipher, considering the 
    26-letter English alphabet.
    
    Parameters
    ----------
    ciphertext: str,
        ciphertext to be decoded
    shift: int, optional (default=0)
        alphabet shift that maps the English alphabet into the cipher alphabet.
    
    Return
    ------
    plaintext: str,
        decoded ciphertext.
    '''
    # define hard-coded 26-letter English alphabet
    alphabet = 'abcdefghijklmnopqrstuvwxyz'
    
    # build a dictionary that maps the plain alphabet into the cipher alphabet
    cipher_alphabet = alphabet[-shift:] + alphabet[:-shift]
    mapping = dict(zip(alphabet, cipher_alphabet))
    
    # iter over each charater of the ciphertext when apply substitution
    # characters not in the alphabet are left unchanged
    plaintext = ''.join([mapping[char] if char in alphabet else char
                         for char in ciphertext.lower()])
    return plaintext

In [23]:
# code snippet to test the implementation of the decoder
ciphertext = 'lipps!' # 'hello!' encoded with shift=4
plaintext = caesar_decoding(ciphertext, shift=4)

print(ciphertext, '->', plaintext)

lipps! -> hello!


### Ciphertext

Then, we need to load the ciphertext stored in the file `ciphertext_caesar.txt`.

We know that:
- ciphertext contains the text of a [Wikipedia](https://www.wikipedia.org/) page encrypted with a Caesar Cipher.
- cipher consider the 26-letter English alphabet
- characters that do not belong to the alphabet (such as numbers and special characters) are left unchanged.

In [25]:
with open('ciphertext_caesar.txt', mode='r', encoding='utf8') as file:
    ciphertext = file.read()
    
# print just first characters
print(ciphertext[:1000])

qfmdhcufodvm, cf qfmdhczcum (tfca obqwsbh ufssy: κρυπτός, fcaobwnsr: yfmdhóg "vwrrsb, gsqfsh"; obr γράφειν ufodvswb, "hc kfwhs", cf -λογία -zcuwo, "ghirm", fsgdsqhwjszm), wg hvs dfoqhwqs obr ghirm ct hsqvbweisg tcf gsqifs qcaaibwqohwcb wb hvs dfsgsbqs ct hvwfr dofhwsg qozzsr orjsfgofwsg. acfs usbsfozzm, qfmdhcufodvm wg opcih qcbghfiqhwbu obr obozmnwbu dfchcqczg hvoh dfsjsbh hvwfr dofhwsg cf hvs dipzwq tfca fsorwbu dfwjohs asggousg; jofwcig ogdsqhg wb wbtcfaohwcb gsqifwhm giqv og roho qcbtwrsbhwozwhm, roho wbhsufwhm, oihvsbhwqohwcb, obr bcb-fsdirwohwcb ofs qsbhfoz hc acrsfb qfmdhcufodvm. acrsfb qfmdhcufodvm slwghg oh hvs wbhsfgsqhwcb ct hvs rwgqwdzwbsg ct aohvsaohwqg, qcadihsf gqwsbqs, szsqhfwqoz sbuwbssfwbu, qcaaibwqohwcb gqwsbqs, obr dvmgwqg. oddzwqohwcbg ct qfmdhcufodvm wbqzirs szsqhfcbwq qcaasfqs, qvwd-pogsr domasbh qofrg, rwuwhoz qiffsbqwsg, qcadihsf doggkcfrg, obr awzwhofm qcaaibwqohwcbg.
qfmdhcufodvm dfwcf hc hvs acrsfb ous kog sttsqhwjszm gmbcbmacig kwhv sbqfmdhwcb, qcbjsfhwbu w

### letters distribution

A piece of knowledge that can be useful to break a Caesar cipher is the distribution of the alphabet letters in English language. It is stored in `alphabet_distribution.mat`.

The distribution of the letters in English language has been estimated by observing many different Wikipedia pages.

In [26]:
# loading the distribution from .mat file
mdict = io.loadmat('alphabet_distribution.mat', squeeze_me=True)
alphabet = mdict['alphabet']
frequency = mdict['frequency']

# print distribution
for letter, freq in zip(alphabet, frequency):
    print(f'{letter}: {freq}')

a: 29158
b: 5724
c: 12570
d: 13592
e: 44376
f: 6835
g: 7067
h: 15209
i: 28203
j: 875
k: 4124
l: 15094
m: 9084
n: 24554
o: 27620
p: 8505
q: 454
r: 21467
s: 22771
t: 34070
u: 11987
v: 3776
w: 6113
x: 883
y: 6987
z: 336


In [17]:
# plot distribution as bar plot
fig, ax = plt.subplots(figsize=(5,3))
probability = frequency/np.sum(frequency)
ax.bar(alphabet, probability)
ax.set(xlabel='letter', ylabel='probability')
ax.grid(True)
fig.tight_layout()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [18]:
# plot distribution by sorting letters by frequency in descending order
fig, ax = plt.subplots(figsize=(5,3))
idx = np.argsort(probability)[::-1]
ax.bar(alphabet[idx], probability[idx])
ax.set(xlabel='letter', ylabel='probability')
ax.grid(True)
fig.tight_layout()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

### Brute Force

Since alphabet is 26 letters long only 26 shifts are possible you can try all possibilities and check them all.

In [27]:
# try every possible shift and print the corresponding plaintext
for shift in range(len(alphabet)):
    plaintext = caesar_decoding(ciphertext, shift=shift)
    print(f'{shift:2d} -> {plaintext[:73]}')

 0 -> qfmdhcufodvm, cf qfmdhczcum (tfca obqwsbh ufssy: κρυπτός, fcaobwnsr: yfmd
 1 -> pelcgbtencul, be pelcgbybtl (sebz napvrag terrx: κρυπτός, ebznavmrq: xelc
 2 -> odkbfasdmbtk, ad odkbfaxask (rday mzouqzf sdqqw: κρυπτός, daymzulqp: wdkb
 3 -> ncjaezrclasj, zc ncjaezwzrj (qczx lyntpye rcppv: κρυπτός, czxlytkpo: vcja
 4 -> mbizdyqbkzri, yb mbizdyvyqi (pbyw kxmsoxd qboou: κρυπτός, bywkxsjon: ubiz
 5 -> lahycxpajyqh, xa lahycxuxph (oaxv jwlrnwc pannt: κρυπτός, axvjwrinm: tahy
 6 -> kzgxbwozixpg, wz kzgxbwtwog (nzwu ivkqmvb ozmms: κρυπτός, zwuivqhml: szgx
 7 -> jyfwavnyhwof, vy jyfwavsvnf (myvt hujplua nyllr: κρυπτός, yvthupglk: ryfw
 8 -> ixevzumxgvne, ux ixevzurume (lxus gtioktz mxkkq: κρυπτός, xusgtofkj: qxev
 9 -> hwduytlwfumd, tw hwduytqtld (kwtr fshnjsy lwjjp: κρυπτός, wtrfsneji: pwdu
10 -> gvctxskvetlc, sv gvctxspskc (jvsq ergmirx kviio: κρυπτός, vsqermdih: ovct
11 -> fubswrjudskb, ru fubswrorjb (iurp dqflhqw juhhn: κρυπτός, urpdqlchg: nubs
12 -> etarvqitcrja, qt etarvqnqia (htqo 

The only shift resulting in a plausible plaintext is `shift`=14.

In [28]:
shift = 14

In [31]:
plaintext = caesar_decoding(ciphertext, shift=shift)
print(plaintext[:1000])

cryptography, or cryptology (from ancient greek: κρυπτός, romanized: kryptós "hidden, secret"; and γράφειν graphein, "to write", or -λογία -logia, "study", respectively), is the practice and study of techniques for secure communication in the presence of third parties called adversaries. more generally, cryptography is about constructing and analyzing protocols that prevent third parties or the public from reading private messages; various aspects in information security such as data confidentiality, data integrity, authentication, and non-repudiation are central to modern cryptography. modern cryptography exists at the intersection of the disciplines of mathematics, computer science, electrical engineering, communication science, and physics. applications of cryptography include electronic commerce, chip-based payment cards, digital currencies, computer passwords, and military communications.
cryptography prior to the modern age was effectively synonymous with encryption, converting i

It make sense!

#### Fitness Measure

The first attempt was successful. Each possible shift was tested and the correspondent plaintext was manually analyzed. Information about alphabet distribution was completely ignored.

> Is it possible to use the alphabet distribution to assess the quality of the decoded plaintext?

Yes, it is. We can use a metric that measures how the letter distribution in the decoded plaintext **fit** the actual letter distribution. The *fitness measure* measures how likely a given plaintext is the right one.

Fitness measure $f$ for a given plaintext $\mathcal{P}$ is defined as follow:
$$ f(\mathcal{P}) = \sum_{a\in \mathcal{P}} a \cdot p_x(a) $$
where:
- $a$ represents each single character of the plaintext $\mathcal{P}$
- $p_x$ identifies the probability distribution of generic letter $x$. Therefore, $p_x(a)$ is the probability to observe the character $a$.

In [33]:
def fitness(text):
    ''' fitness measure '''
    dist = dict(zip(alphabet, probability)) # hard-coded alphabet distribution
    measure = np.sum([text.count(char)*dist[char] for char in alphabet])
    return measure

In [34]:
# compute fitness for each possible plaintext
fit = np.array([fitness(caesar_decoding(ciphertext, shift=shift))
                for shift in range(len(alphabet))])

# get shift with maximum fitness measure
imax = np.argmax(fit)
print(f'best fitness: shift={imax}')

# plot fitness measure for all possible plaintext and mark maximum
fig, ax = plt.subplots(figsize=(5,3))
ax.bar(np.arange(len(alphabet)), fit)
ax.plot(imax, fit[imax], '*', color='C3')
ax.set(xlabel='shift', ylabel='fitness measure')
ax.grid(True)
fig.tight_layout()

best fitness: shift=14


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

### Frequency Analysis

Knowing what is the typical frequency for each letter (e.g., in English, letters «e», «t», «a», «i» are more common than others), it is possible to infer what shift was used to encrypt the plaintext. We just need to oserve the frequency of the characters in the ciphertext and find what shift best match the two distribution.

In [35]:
# computing alphabet distribution on the ciphertext
freq_ciphertext = np.array([ciphertext.count(x) for x in alphabet])
prob_ciphertext = freq_ciphertext/len(ciphertext)

# print letters frequency and estimated probability
for letter, freq, prob in zip(alphabet, freq_ciphertext, prob_ciphertext):
    print(f'{letter}: {freq:5d} ({prob:.3f})')

a:  1078 (0.023)
b:  2541 (0.054)
c:  2519 (0.053)
d:  1268 (0.027)
e:    83 (0.002)
f:  2656 (0.056)
g:  2595 (0.055)
h:  3425 (0.072)
i:   990 (0.021)
j:   397 (0.008)
k:   405 (0.009)
l:   115 (0.002)
m:  1141 (0.024)
n:    33 (0.001)
o:  3048 (0.064)
p:   572 (0.012)
q:  1881 (0.040)
r:  1242 (0.026)
s:  4614 (0.097)
t:   758 (0.016)
u:   757 (0.016)
v:  1630 (0.034)
w:  2947 (0.062)
x:    21 (0.000)
y:   314 (0.007)
z:  1578 (0.033)


In [36]:
fig, ax = plt.subplots(2, 1, figsize=(5,4))

ax[0].bar(alphabet, probability, label='plaintext')
ax[0].set(xlabel='letter', ylabel='probability')
ax[0].grid(True)
ax[0].legend(loc='upper right')

ax[1].bar(alphabet, prob_ciphertext, label='ciphertext')
ax[1].set(xlabel='letter', ylabel='probability')
ax[1].grid(True)
ax[1].legend(loc='upper right')

fig.tight_layout()


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

By comparing the two distribution plots, It is evident that the letter `e` (the most common) is mapped to the letter `s`, `f` to `t` and so on. The shift used by the Caesar cipher is therefore 14.

In [37]:
shift = 14

In [38]:
plaintext = caesar_decoding(ciphertext, shift=shift)
print(plaintext[:1000])

cryptography, or cryptology (from ancient greek: κρυπτός, romanized: kryptós "hidden, secret"; and γράφειν graphein, "to write", or -λογία -logia, "study", respectively), is the practice and study of techniques for secure communication in the presence of third parties called adversaries. more generally, cryptography is about constructing and analyzing protocols that prevent third parties or the public from reading private messages; various aspects in information security such as data confidentiality, data integrity, authentication, and non-repudiation are central to modern cryptography. modern cryptography exists at the intersection of the disciplines of mathematics, computer science, electrical engineering, communication science, and physics. applications of cryptography include electronic commerce, chip-based payment cards, digital currencies, computer passwords, and military communications.
cryptography prior to the modern age was effectively synonymous with encryption, converting i

## Simple Substitution Cipher

In a simple substitution cipher, every plaintext character is replaced with a different ciphertext character. 

As for Caesar Cipher, plaintext and ciphertext share the same set of characters (the alphabet), but, mapping from plaintext to ciphertext can be any of the $26! \sim 10^{26}\sim 2^{88}$ possibilities

Since nowadays machines cannot explore 26! candidates, **frequency analysis** must be exploited to narrow down their number.

### Decoder

As for Caesar cipher the decoder must know the **alphabet**, but this time a bare shift is not sufficient to define a decoder but there is the need of a **mapping rule** that maps each character of the plaintext to the character of the ciphertext.

Again, we can implement the decoder as a function that takes the `ciphertext`a and the mapping `rule` as input and returns the `plaintext`. The alphabet is hard-coded in the function.

In [39]:
def simple_decoding(ciphertext, rule):
    '''
    Decode a ciphertext encrypted with a Simple Substitution Cipher, 
    considering the 26-letter English alphabet.
    
    Parameters
    ----------
    ciphertext: str,
        ciphertext to be decoded
    rule: dict,
        map from cipher alphabet to plaintext alphabet.
    
    Return
    ------
    plaintext: str,
        decoded ciphertext.
    '''
    plaintext = ''.join([rule[char] if char in alphabet else char
                         for char in ciphertext.lower()])
    return plaintext

### Ciphertext

Let us load the ciphertext stored in the file `ciphertext_simple.txt`.

As before, we know that:
- ciphertext contains the text of a [Wikipedia](https://www.wikipedia.org/) page encrypted with a Simple Substitution Cipher.
- cipher consider the 26-letter English alphabet
- characters that do not belong to the alphabet (such as numbers and special characters) are left unchanged.

In [40]:
with open('ciphertext_simple.txt', mode='r', encoding='utf8') as file:
    ciphertext = file.read()
print(ciphertext[:1000])

thycbr rhdffb wmyeefe (ynvxh 30, 1916 – iravcyvo 24, 2001) dyw ye ylrvxtye lyjmrlyjxtxye, rhrtjvxtyh resxerrv, yeb tvonjfsvynmrv qefde yw "jmr iyjmrv fi xeifvlyjxfe jmrfvo". wmyeefe xw efjrb ifv myzxes ifcebrb xeifvlyjxfe jmrfvo dxjm y hyeblyvq nynrv, "y lyjmrlyjxtyh jmrfvo fi tfllcextyjxfe", dmxtm mr ncahxwmrb xe 1948.
mr xw yhwf drhh qefde ifv ifcebxes bxsxjyh txvtcxj brwxse jmrfvo xe 1937, dmre—yw y 21-oryv-fhb lywjrv'w brsvrr wjcbrej yj jmr lywwytmcwrjjw xewjxjcjr fi jrtmefhfso (lxj)—mr dvfjr mxw jmrwxw brlfewjvyjxes jmyj rhrtjvxtyh ynnhxtyjxfew fi affhrye yhsravy tfchb tfewjvctj yeo hfsxtyh eclrvxtyh vrhyjxfewmxn. wmyeefe tfejvxacjrb jf jmr ixrhb fi tvonjyeyhowxw ifv eyjxfeyh brirewr bcvxes dfvhb dyv xx, xethcbxes mxw icebylrejyh dfvq fe tfbravryqxes yeb wrtcvr jrhrtfllcextyjxfew.


== axfsvynmo ==


=== tmxhbmffb ===
jmr wmyeefe iylxho hxzrb xe syohfvb, lxtmxsye, yeb thycbr dyw afve xe y mfwnxjyh xe eryvao nrjfwqro. mxw iyjmrv, thycbr wv. (1862–1934) dyw y acwxerwwlye yeb ifv y d

### Frequency Analysis

For reasonably large pieces of text (with enough characters to be statistically relevant), a possible procedure can be:
 - to just replace the most common ciphertext character with the most common character in the plaintext (for English text is `e`). 
 - to replace the second most common ciphertext character with the second most common character in the plaintext 
 - and so on

In [41]:
text = ''.join(filter(lambda x: x in alphabet, ciphertext))
print(text[:1000])

thycbrrhdffbwmyeefeynvxhiravcyvodywyeylrvxtyelyjmrlyjxtxyerhrtjvxtyhresxerrvyebtvonjfsvynmrvqefdeywjmriyjmrvfixeifvlyjxfejmrfvowmyeefexwefjrbifvmyzxesifcebrbxeifvlyjxfejmrfvodxjmyhyeblyvqnynrvylyjmrlyjxtyhjmrfvofitfllcextyjxfedmxtmmrncahxwmrbxemrxwyhwfdrhhqefdeifvifcebxesbxsxjyhtxvtcxjbrwxsejmrfvoxedmreywyoryvfhblywjrvwbrsvrrwjcbrejyjjmrlywwytmcwrjjwxewjxjcjrfijrtmefhfsolxjmrdvfjrmxwjmrwxwbrlfewjvyjxesjmyjrhrtjvxtyhynnhxtyjxfewfiaffhryeyhsravytfchbtfewjvctjyeohfsxtyheclrvxtyhvrhyjxfewmxnwmyeefetfejvxacjrbjfjmrixrhbfitvonjyeyhowxwifveyjxfeyhbrirewrbcvxesdfvhbdyvxxxethcbxesmxwicebylrejyhdfvqfetfbravryqxesyebwrtcvrjrhrtfllcextyjxfewaxfsvynmotmxhbmffbjmrwmyeefeiylxhohxzrbxesyohfvblxtmxsyeyebthycbrdywafvexeymfwnxjyhxeeryvaonrjfwqromxwiyjmrvthycbrwvdywyacwxerwwlyeyebifvydmxhryucbsrfinvfayjrxesyohfvbmxwlfjmrvlyarhdfhiwmyeefedywyhyescysrjrytmrvdmfyhwfwrvzrbywjmrnvxetxnyhfisyohfvbmxsmwtmffhthycbrwvdywybrwtrebyejfierdurvwrowrjjhrvwdmxhrlyarhdywytmxhbfisrvlyexllxsvyejwlfwjfijmrixvwjoryvwfiwmyeefe

In [42]:
# computing alphabet distribution on the ciphertext
freq_ciphertext = np.array([text.count(x) for x in alphabet])
prob_ciphertext = freq_ciphertext/len(text)

# print letters frequency and estimated probability
for letter, freq, prob in zip(alphabet, freq_ciphertext, prob_ciphertext):
    print(f'{letter}: {freq:5d} ({prob:.3f})')

a:   269 (0.015)
b:   648 (0.037)
c:   454 (0.026)
d:   283 (0.016)
e:  1571 (0.090)
f:  1382 (0.080)
g:    10 (0.001)
h:   740 (0.043)
i:   347 (0.020)
j:  1410 (0.081)
k:    16 (0.001)
l:   504 (0.029)
m:   847 (0.049)
n:   354 (0.020)
o:   289 (0.017)
p:    23 (0.001)
q:    82 (0.005)
r:  1912 (0.110)
s:   327 (0.019)
t:   689 (0.040)
u:    25 (0.001)
v:  1001 (0.058)
w:  1134 (0.065)
x:  1388 (0.080)
y:  1506 (0.087)
z:   166 (0.010)


In [43]:
fig, ax = plt.subplots(2, 1, figsize=(5,4))

ax[0].bar(alphabet, probability, label='plaintext')
ax[0].set(xlabel='letter', ylabel='probability')
ax[0].grid(True)
ax[0].legend(loc='upper right')

ax[1].bar(alphabet, prob_ciphertext, label='ciphertext')
ax[1].set(xlabel='letter', ylabel='probability')
ax[1].grid(True)
ax[1].legend(loc='upper right')

fig.tight_layout()


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Let us plot the two distribution in descending order so that we can match the ciphertext most common character with the one that is most common in a typical Eanglish Wikipedia page.

In [44]:
fig, ax = plt.subplots(2, 1, figsize=(5,4))

idx_plain = np.argsort(probability)[::-1] # sorted indexes for plaintext
ax[0].bar(alphabet[idx_plain], probability[idx_plain], label='plaintext')
ax[0].set(xlabel='letter', ylabel='probability')
ax[0].grid(True)
ax[0].legend(loc='upper right')

idx_cipher = np.argsort(prob_ciphertext)[::-1] # sorted indexes for ciphertext
ax[1].bar(alphabet[idx_cipher], prob_ciphertext[idx_cipher], 
          label='ciphertext')
ax[1].set(xlabel='letter', ylabel='probability')
ax[1].grid(True)
ax[1].legend(loc='upper right')

fig.tight_layout()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Let us define our first guess by matching the two distributions.

In [45]:
rule = dict(zip(alphabet[idx_cipher], alphabet[idx_plain]))

# visualize the mapping
idx = np.argsort(list(rule.values()))
print(' plain alphabet:', ''.join(np.array(list(rule.values()))[idx]))
print('cipher alphabet:', ''.join(np.array(list(rule.keys()))[idx]))

 plain alphabet: abcdefghijklmnopqrstuvwxyz
cipher alphabet: yabtroimjpzhcfxnkvwelqdusg


In [46]:
plaintext = simple_decoding(ciphertext, rule)
print(plaintext[:700])

dlamce elwnnc shattnt (aprol 30, 1916 – gebrmarf 24, 2001) was at auerodat uaiheuaiodoat, eledirodal etyoteer, atc drfpinyrapher vtnwt as "ihe gaiher ng otgnruaiont ihenrf". shattnt os tniec gnr hakoty gnmtcec otgnruaiont ihenrf woih a latcuarv paper, "a uaiheuaiodal ihenrf ng dnuumtodaiont", whodh he pmbloshec ot 1948.
he os alsn well vtnwt gnr gnmtcoty coyoial dordmoi cesoyt ihenrf ot 1937, whet—as a 21-fear-nlc uasier's ceyree simceti ai ihe uassadhmseiis otsioimie ng iedhtnlnyf (uoi)—he wrnie hos ihesos ceuntsiraioty ihai eledirodal applodaionts ng bnnleat alyebra dnmlc dntsirmdi atf lnyodal tmuerodal relaiontshop. shattnt dntirobmiec in ihe goelc ng drfpiatalfsos gnr taiontal cegetse cm


As expected, the first guess is not perfect. However, we can start recognizing some words such as "aprol" as first word inside the parenthesis that probably is a "april". This means that the letter mapped to `o` should be mapped to `i`. 

Let us modify the rule and check again.

In [47]:
rule['x'], rule['j'] = 'i', 'o'

# visualize the mapping
idx = np.argsort(list(rule.values()))
print(' plain alphabet:', ''.join(np.array(list(rule.values()))[idx]))
print('cipher alphabet:', ''.join(np.array(list(rule.keys()))[idx]))

 plain alphabet: abcdefghijklmnopqrstuvwxyz
cipher alphabet: yabtroimxpzhcfjnkvwelqdusg


In [48]:
plaintext = simple_decoding(ciphertext, rule)
print(plaintext[:700])

dlamce elwnnc shattnt (april 30, 1916 – gebrmarf 24, 2001) was at aueridat uaoheuaoidiat, eledoridal etyiteer, atc drfponyrapher vtnwt as "ohe gaoher ng itgnruaoint ohenrf". shattnt is tnoec gnr hakity gnmtcec itgnruaoint ohenrf wioh a latcuarv paper, "a uaoheuaoidal ohenrf ng dnuumtidaoint", whidh he pmblishec it 1948.
he is alsn well vtnwt gnr gnmtcity ciyioal dirdmio cesiyt ohenrf it 1937, whet—as a 21-fear-nlc uasoer's ceyree somceto ao ohe uassadhmseoos itsoiomoe ng oedhtnlnyf (uio)—he wrnoe his ohesis ceuntsoraoity ohao eledoridal applidaoints ng bnnleat alyebra dnmlc dntsormdo atf lnyidal tmueridal relaointship. shattnt dntoribmoec on ohe gielc ng drfpoatalfsis gnr taointal cegetse cm


Assuming that inside the parenthesis there is a date, the word "gebrmarf" probably corresponds to "february". This means that `e`, `b`, `r`, `a` are correctly mapped but `g`, `m`, `f` are not.

With this iterative and manual procedure we can get to the final solution.

In [49]:
rule['x'], rule['j'] = 'i', 'o'
rule['i'], rule['o'], rule['s'] = 'f', 'y', 'g'
rule['c'], rule['l'] = 'u', 'm'
rule['e'], rule['j'], rule['f'] = 'n', 't', 'o'
rule['b'], rule['t'] = 'd', 'c'
rule['z'], rule['q'] = 'v', 'k'
rule['p'], rule['u'] = 'x', 'j'
rule['k'], rule['g'] = 'z', 'q'

idx = np.argsort(list(rule.values()))
print(''.join(np.array(list(rule.values()))[idx]))
print(''.join(np.array(list(rule.keys()))[idx]))

abcdefghijklmnopqrstuvwxyz
yatbrismxuqhlefngvwjczdpok


In [50]:
plaintext = simple_decoding(ciphertext, rule)
print(plaintext[:1200])

claude elwood shannon (april 30, 1916 – february 24, 2001) was an american mathematician, electrical engineer, and cryptographer known as "the father of information theory". shannon is noted for having founded information theory with a landmark paper, "a mathematical theory of communication", which he published in 1948.
he is also well known for founding digital circuit design theory in 1937, when—as a 21-year-old master's degree student at the massachusetts institute of technology (mit)—he wrote his thesis demonstrating that electrical applications of boolean algebra could construct any logical numerical relationship. shannon contributed to the field of cryptanalysis for national defense during world war ii, including his fundamental work on codebreaking and secure telecommunications.


== biography ==


=== childhood ===
the shannon family lived in gaylord, michigan, and claude was born in a hospital in nearby petoskey. his father, claude sr. (1862–1934) was a businessman and for a w